In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
sys.path.insert(0, "../")
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import MobileNetV2

from models import ModelWrapper_TorchVisionNet
from dataset import VoiceAntiSpoofDataset
from utils import read_fromBaseline
#import DftSpectrogram
from librosa.feature import mfcc


from kekas import Keker, DataOwner
from kekas.callbacks import Callback



/opt/conda/envs/idrnd/lib/python3.6/site-packages/kekas/keker.py:9: UserWarning: Error 'No module named 'apex''' during importing apex library. To use mixed precison you should install it from https://github.com/NVIDIA/apex
  warnings.warn(f"Error '{e}'' during importing apex library. To use mixed precison"


In [2]:
MN2 = MobileNetV2()
MN2.classifier[1] = nn.Linear(1280, 2, bias=False)
model = ModelWrapper_TorchVisionNet(MN2).to('cuda')
sum(p.numel() for p in model.parameters() if p.requires_grad)

2226438

In [4]:
dataset_dir = '/home/data/datasets/idrnd/Training_Data/'
dataset = VoiceAntiSpoofDataset(dataset_dir, 'all', read_fromBaseline, transform=[lambda x: mfcc(x),  
                                lambda x: x[None, ...].astype(np.float32)])
batch_size = 64
num_workers = 4
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)


In [5]:
dataowner = DataOwner(dataloader, None, None)
weights = torch.tensor([5, 1.25])
weights = weights / weights.sum()
weights = weights.to('cuda')
criterion = nn.CrossEntropyLoss(weight=weights)

In [6]:
def step_fn(model: torch.nn.Module,
            batch: torch.Tensor) -> torch.Tensor:
    """Determine what your model will do with your data.

    Args:
        model: the pytorch module to pass input in
        batch: the batch of data from the DataLoader

    Returns:
        The models forward pass results
    """
    
    voice = batch['data']
    return model(voice)

        

In [7]:
keker = Keker(model=model,
              dataowner=dataowner,
              criterion=criterion,
              step_fn=step_fn,                    # previosly defined step function
              target_key="label",                 # remember, we defined it in the reader_fn for DataKek? 
              opt=torch.optim.Adam,               # optimizer class. if note specifiyng, 
                                                  # an SGD is using by default
              opt_params={"weight_decay": 1e-5})  # optimizer kwargs in dict format (optional too)

In [ ]:
keker.kek(lr=1e-3,
          epochs=50,
          sched=torch.optim.lr_scheduler.MultiStepLR,       # pytorch lr scheduler class
          sched_params={"milestones":torch.tensor([25, 40]), "gamma": 0.1,})  # schedulres kwargas in dict format

Epoch 1/50:   0% 0/782 [00:00<?, ?it/s]

In [ ]:
%debug